In [13]:
import json
import pandas as pd
import altair as alt
import os


# Function to extract relevant properties for the Ragone plot
def extract_properties(data):
    try:
        properties = {prop['@type'][0]: prop['hasNumericalPart']['hasNumericalValue'] for prop in data['hasProperty']}
        properties['Name'] = data['schema:name']
        properties['Manufacturer'] = data['schema:manufacturer']['schema:name']
        # Extract schema:subjectOf if available
        if 'schema:subjectOf' in data:
            properties['References'] = ", ".join([subject['@id'] for subject in data['schema:subjectOf']])
        else:
            properties['References'] = 'N/A'
        if 'hasPositiveElectrode' in data:
            cathode_type = str(data['hasPositiveElectrode']['hasActiveMaterial']['@type'])
            # Replace long names with abbreviations
            cathode_type = cathode_type.replace('LithiumIronPhosphate', 'LFP')
            cathode_type = cathode_type.replace('LithiumNickelManganeseCobaltOxide', 'NMC')
            cathode_type = cathode_type.replace('LithiumNickelCobaltAluminiumOxide', 'NCA')
            properties['Cathode'] = cathode_type
        else:
            properties['Cathode'] = 'N/A'
        return properties
    except KeyError:
        return None

In [14]:

# Define the directory containing the JSON files
directory = 'BatteryTypeJson'

# List all JSON files in the directory
json_files = [file for file in os.listdir(directory) if file.endswith('.json')]

# Initialize an empty list to store properties from all JSON files
all_properties = []


# Process each JSON file
for file in json_files:
    file_path = os.path.join(directory, file)
    with open(file_path) as f:
        data = json.load(f)
    properties = extract_properties(data)
    if properties:
        all_properties.append(properties)

# Create a DataFrame
df = pd.DataFrame(all_properties)

# Filter out rows without sufficient information for Specific Energy and Specific Power calculation
df = df.dropna(subset=['NominalVoltage', 'RatedCapacity', 'Mass', 'MaximumContinuousDischargeCurrent'])

# Add Specific Energy and Specific Power calculations
df['Specific Energy (Wh/kg)'] = (df['NominalVoltage'] * df['RatedCapacity']) / df['Mass']
df['Specific Power (W/kg)'] = (df['NominalVoltage'] * df['MaximumContinuousDischargeCurrent']) / df['Mass']

df.head()


,RatedCapacity,CycleLife,NominalVoltage,UpperVoltageLimit,LowerVoltageLimit,DischargeCurrent,MaximumContinuousDischargeCurrent,Mass,ChargingCurrent,Height,Width,Length,Name,Manufacturer,References,Cathode,Diameter,Specific Energy (Wh/kg),Specific Power (W/kg)
0,100.0,7000.0,3.30,3.65,2.5,50.0,300.0,3.500,50.0,0.0610,0.1450,0.220,WB-LYP100AHA,Thunder-Sky,https://www.semanticscholar.org/paper/d86ac0b0...,LFP,NaN,94.285714,282.857143
1,7.5,3000.0,3.20,3.65,2.5,NaN,37.5,0.195,NaN,0.0079,0.0980,0.130,7799130P,Amita Technologies,N/A,LFP,NaN,123.076923,615.384615
2,26.0,NaN,3.65,4.20,2.7,5.2,26.0,0.550,5.2,0.0075,0.1610,0.227,A123 26AH,A123,N/A,NMC,NaN,172.545455,172.545455
3,202.0,3000.0,3.20,3.65,2.5,101.0,202.0,3.900,101.0,0.0540,0.1740,0.210,LP54173210-202Ah,Lishen,N/A,LFP,NaN,165.743590,165.743590
4,34.0,15000.0,2.20,2.80,1.5,34.0,204.0,1.080,34.0,0.0120,0.1785,0.286,LT 34 Ah (936901),LECLANCHÉ,N/A,LithiumTitanate,NaN,69.259259,415.555556


In [30]:
# Define selection for Cathode and Manufacturer
cathode_selection = alt.selection_point(fields=['Cathode'], bind='legend')
#manufacturer_selection = alt.selection_point(fields=['Manufacturer'], bind='legend')

# Create the Ragone plot using Altair
ragone_plot = alt.Chart(df).mark_circle(size=60).encode(
    x=alt.X('Specific Energy (Wh/kg)', title='Specific Energy (Wh/kg)'),
    y=alt.Y('Specific Power (W/kg)', scale=alt.Scale(type='log'), title='Specific Power (W/kg)'),
    color=alt.condition(cathode_selection, 'Cathode:N', alt.value('lightgray')),
    opacity=alt.condition(cathode_selection, alt.value(1), alt.value(0.2)),
    tooltip=['Name', 'Manufacturer', 'RatedCapacity', 'NominalVoltage','Mass','MaximumContinuousDischargeCurrent','ChargingCurrent', 'CycleLife','Cathode','References']
).properties(
    width=700,
    height=400,
    title='Ragone Plot' 
).add_params(
    cathode_selection
).interactive()

# Show the plot
ragone_plot.show()

alt.Chart(...)

In [31]:
# Define selection for Cathode and Manufacturer
cathode_selection = alt.selection_point(fields=['Cathode'], bind='legend')
#manufacturer_selection = alt.selection_point(fields=['Manufacturer'], bind='legend')

# Create the Ragone plot using Altair
ragone_plot = alt.Chart(df).mark_circle(size=60).encode(
    x=alt.X('Specific Energy (Wh/kg)', title='Specific Energy (Wh/kg)'),
    y=alt.Y('Specific Power (W/kg)', scale=alt.Scale(type='log'), title='Specific Power (W/kg)'),
    color=alt.condition(cathode_selection, 'Cathode:N', alt.value('lightgray')),
    opacity=alt.condition(cathode_selection, alt.value(1), alt.value(0.2)),
    tooltip=['Name', 'Manufacturer', 'RatedCapacity', 'NominalVoltage','Mass','MaximumContinuousDischargeCurrent','ChargingCurrent', 'CycleLife','Cathode','References']
).properties(
    width=700,
    height=400,
    title='Ragone Plot' 
).transform_filter(
    alt.FieldRangePredicate(field='RatedCapacity', range=[3, 4])
    #alt.FieldOneOfPredicate(field='Manufacturer', oneOf=['LG Chem','Samsung','Panasonic'])
).add_params(
    cathode_selection
).interactive()

# Show the plot
ragone_plot.show()

alt.Chart(...)

In [29]:
# Define selection for Cathode and Manufacturer
#cathode_selection = alt.selection_point(fields=['Cathode'], bind='legend')
manufacturer_selection = alt.selection_point(fields=['Manufacturer'], bind='legend')

# Create the Ragone plot using Altair
ragone_plot = alt.Chart(df).mark_circle(size=60).encode(
    x=alt.X('Specific Energy (Wh/kg)', title='Specific Energy (Wh/kg)'),
    y=alt.Y('Specific Power (W/kg)', scale=alt.Scale(type='log'), title='Specific Power (W/kg)'),
    color=alt.condition(manufacturer_selection, 'Manufacturer:N', alt.value('lightgray')),
    opacity=alt.condition(manufacturer_selection, alt.value(1), alt.value(0.2)),
    tooltip=['Name', 'Manufacturer', 'RatedCapacity', 'NominalVoltage','Mass','MaximumContinuousDischargeCurrent','ChargingCurrent', 'CycleLife','Cathode','References']
).properties(
    width=700,
    height=400,
    title='Ragone Plot' 
).transform_filter(
    alt.FieldOneOfPredicate(field='Manufacturer', oneOf=['LG Chem','Samsung','Panasonic'])
).add_params(
    manufacturer_selection
).interactive()

# Show the plot
ragone_plot.show()

alt.Chart(...)

In [33]:
# Define selection for Cathode and Manufacturer
cathode_selection = alt.selection_point(fields=['Cathode'], bind='legend')
#manufacturer_selection = alt.selection_point(fields=['Manufacturer'], bind='legend')

# Create the Ragone plot using Altair
ragone_plot = alt.Chart(df[df['References'] != 'N/A']).mark_circle(size=60).encode(
    x=alt.X('Specific Energy (Wh/kg)', title='Specific Energy (Wh/kg)'),
    y=alt.Y('Specific Power (W/kg)', scale=alt.Scale(type='log'), title='Specific Power (W/kg)'),
    color=alt.condition(cathode_selection, 'Cathode:N', alt.value('lightgray')),
    opacity=alt.condition(cathode_selection, alt.value(1), alt.value(0.2)),
    tooltip=['Name', 'Manufacturer', 'RatedCapacity', 'NominalVoltage','Mass','MaximumContinuousDischargeCurrent','ChargingCurrent', 'CycleLife','Cathode','References']
).properties(
    width=700,
    height=400,
    title='Ragone Plot with Literature' 
).add_params(
    cathode_selection
).interactive()

# Show the plot
ragone_plot.show()

alt.Chart(...)